
# `upstox_v3_18Sep_scalper_llm_v3recon_full.ipynb`
**NIFTY Intraday Scalper (Live‑First)** — Spot‑Anchored + Greeks + IV‑Z + Marketable‑Limit + Recenter + Latency + **LLM Router** + **PnL Ledger** + **Multi‑Entry** + **V3 Exec & Live Reconciler**.


In [ ]:

import os, json, time, threading, math, traceback, uuid, re
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
import numpy as np
import pandas as pd
from IPython.display import display

pd.set_option("display.width", 160)
pd.set_option("display.max_columns", 120)

# Upstox SDK
try:
    import upstox_client
    from upstox_client.rest import ApiException
    UPSDK_AVAILABLE = True
except Exception as e:
    UPSDK_AVAILABLE = False
    print("Upstox SDK not available. Install it to run live streaming and orders.")
    print("Example: pip install upstox-python-sdk   # confirm exact name per Upstox docs")


In [ ]:

# ---- Toggles & constants ----
SIMULATION_MODE = False        # This is live-first; set to True only for offline tests
USE_LLM = True
USE_ROUTER = True

ORDERS_LIVE = False            # If True, use V3 place/modify/cancel
EXIT_MANAGER_LIVE = False      # If True, send exits via V3
RECONCILE_LIVE_PNL = True      # Start Portfolio Stream Feed reconciler

UNDERLYING = "NIFTY"
UNDERLYING_SPOT_TOKEN = "NSE_INDEX|Nifty 50"
SPAN_STRIKES = 2
WEBSOCKET_MODE = "full_d30"
IST = "Asia/Kolkata"

STRIKE_STEP = {"NIFTY": 50, "BANKNIFTY": 100, "FINNIFTY": 50}
MAX_QTY_PER_LEG = 300
MAX_OPEN_LEGS = 6

RECENTER_COOLDOWN_S = 60.0
RECENTER_LOG = True

# Scalper gates
DELTA_MIN, DELTA_MAX = 0.45, 0.55
SPREAD_MAX = 0.20
DEPTH_IMB_MIN = 0.15
IV_Z_MAX = 2.0
IV_Z_MIN_COUNT = 30

# Execution
USE_MARKETABLE_LIMITS = True
LIMIT_BUFFER_TICKS = 1
ORDER_MIN_GAP_MS = 200

# LLM + Router
LLM_ENDPOINT = os.getenv("LLM_ENDPOINT", "")  # fast stub endpoint
LLM_TIMEOUT_S = float(os.getenv("LLM_TIMEOUT_S", "0.12"))
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "http://127.0.0.1:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "")  # e.g., "mistral"
OLLAMA_TIMEOUT_S = float(os.getenv("OLLAMA_TIMEOUT_S", "0.20"))
OLLAMA_NUM_PREDICT = int(os.getenv("OLLAMA_NUM_PREDICT", "16"))
LLM_SCORE_THRESHOLD = 0.55
STUB_LOWER, STUB_UPPER = 0.35, 0.75

# Latency logs
LATENCY_LOG = []; LATENCY_LOG_MAX = 5000
ROUTER_LOG = []; ROUTER_LOG_MAX = 5000


In [ ]:

class CredentialUpstox:
    ACCESS_TOKEN = os.getenv("UPSTOX_ACCESS_TOKEN", "")

PRODUCT_MAP = {"MIS": "I", "NRML": "D"}
DEFAULT_PRODUCT = "MIS"


In [ ]:

def to_ist_ms(ms) -> pd.Timestamp:
    try:
        return pd.to_datetime(int(ms), unit="ms", utc=True).tz_convert(IST)
    except Exception:
        return pd.NaT

def resolve_next_listed_expiry(df_instruments: pd.DataFrame, underlying: str, today=None) -> str:
    t = pd.Timestamp.now(IST).normalize() if today is None else pd.Timestamp(today, tz=IST).normalize()
    dfx = df_instruments[(df_instruments["segment"] == "NSE_FO") &
                         (df_instruments["name"].str.upper() == underlying.upper()) &
                         (df_instruments["instrument_type"].isin(["CE","PE"]))].copy()
    if dfx.empty: raise ValueError(f"No derivatives found for {underlying} in instruments master")
    dfx["_exp"] = pd.to_datetime(dfx["expiry"], errors="coerce")
    dfx = dfx[dfx["_exp"] >= t.tz_localize(None)]
    if dfx.empty: raise ValueError(f"No upcoming expiry >= {t.date()} for {underlying}")
    return dfx["_exp"].min().strftime("%Y-%m-%d")

def nearest_strikes_from_spot(spot_ltp: float, underlying: str, span: int = 2):
    step = STRIKE_STEP.get(underlying.upper(), 50)
    nearest = int(step * math.floor((spot_ltp + step/2) / step))
    return [nearest + i * step for i in range(-span, span+1)]

def get_upstox_quote_client():
    if not UPSDK_AVAILABLE: raise RuntimeError("Upstox SDK is not available.")
    if not CredentialUpstox.ACCESS_TOKEN: raise RuntimeError("ACCESS_TOKEN missing. Set UPSTOX_ACCESS_TOKEN.")
    configuration = upstox_client.Configuration(); configuration.access_token = CredentialUpstox.ACCESS_TOKEN
    return upstox_client.MarketQuoteV3Api(upstox_client.ApiClient(configuration))

def _extract_ltp_from_entry(entry: dict) -> float:
    if not isinstance(entry, dict): raise KeyError("Invalid LTP entry")
    for k in ("ltp","last_traded_price","last","close","price"):
        if k in entry and entry[k] is not None: return float(entry[k])
    if "ltpc" in entry and isinstance(entry["ltpc"], dict) and "ltp" in entry["ltpc"]:
        return float(entry["ltpc"]["ltp"])
    raise KeyError(f"No LTP field found in entry keys={list(entry.keys())}")

def get_index_spot_ltp(instrument_key: str = None) -> float:
    instrument_key = instrument_key or UNDERLYING_SPOT_TOKEN
    api = get_upstox_quote_client()
    try:
        api_response = api.get_ltp(instrument_key=[instrument_key] if not isinstance(instrument_key, list) else instrument_key)
        data_dict = api_response.to_dict() if hasattr(api_response, "to_dict") else dict(api_response)
        data = data_dict.get("data", {})
        entry = data.get(instrument_key) or (next(iter(data.values())) if data else {})
        ltp = _extract_ltp_from_entry(entry)
        if not np.isfinite(ltp): raise RuntimeError(f"LTP non-finite: {ltp}")
        return float(ltp)
    except ApiException as e:
        raise RuntimeError(f"Upstox get_ltp ApiException: {e}")


In [ ]:

def load_instruments_live() -> pd.DataFrame:
    url = "https://assets.upstox.com/market-quote/instruments/exchange/NSE.json.gz"
    try:
        df = pd.read_json(url)
    except Exception as e:
        raise RuntimeError(f"Failed to load instruments from {url}: {e}")
    if "expiry" in df:
        exp = pd.to_datetime(df["expiry"], unit="ms", errors="coerce")
        mask = exp.isna() & df["expiry"].notna()
        if mask.any():
            exp2 = pd.to_datetime(df.loc[mask, "expiry"], errors="coerce")
            exp.loc[mask] = exp2
        df["expiry"] = exp.dt.strftime("%Y-%m-%d")
    for col in ("strike_price","lot_size","tick_size","minimum_lot"):
        if col in df: df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

df_futureOptions = load_instruments_live()
expiry_target = resolve_next_listed_expiry(df_futureOptions, UNDERLYING)
df_chain = df_futureOptions[(df_futureOptions["segment"]=="NSE_FO") &
                            (df_futureOptions["name"].str.upper()==UNDERLYING.upper()) &
                            (df_futureOptions["instrument_type"].isin(["CE","PE"])) &
                            (df_futureOptions["expiry"]==expiry_target)].copy()
assert not df_chain.empty, f"No options for {UNDERLYING} {expiry_target}"

try:
    spot_ltp_initial = get_index_spot_ltp(UNDERLYING_SPOT_TOKEN)
    print(f"NIFTY 50 spot LTP: {spot_ltp_initial:.2f}")
except Exception as e:
    print("Spot LTP lookup failed; fallback to chain median:", e)
    spot_ltp_initial = float(df_chain["strike_price"].median())

strike_list = nearest_strikes_from_spot(spot_ltp_initial, UNDERLYING, span=SPAN_STRIKES)
df_chain_sel = df_chain[df_chain["strike_price"].isin(strike_list)].sort_values(["strike_price","instrument_type"])
token_list = df_chain_sel["instrument_key"].dropna().astype(str).unique().tolist()
print(f"Selected strikes from spot {spot_ltp_initial:.2f}: {sorted(set(strike_list))}")
display(df_chain_sel.head(8))


In [ ]:

df_feed = pd.DataFrame(columns=[
    "Token","Ltp","Ltq","Cp",
    "BidP1","BidQ1","AskP1","AskQ1",
    "Ltt","Oi","Iv","Atp","Tbq","Tsq",
    "Delta","Theta","Gamma","Vega","Rho"
])
df_feed_enriched = pd.DataFrame()
_df_lock = threading.Lock()

def enrich_feed(_df_feed: pd.DataFrame, _df_meta: pd.DataFrame, cols_to_add=None) -> pd.DataFrame:
    if cols_to_add is None:
        cols_to_add = ["lot_size","trading_symbol","strike_price","tick_size","instrument_type","expiry","name"]
    left = _df_feed.copy()
    right = _df_meta[["instrument_key"] + [c for c in cols_to_add if c in _df_meta.columns]].drop_duplicates("instrument_key")
    out = left.merge(right, left_on="Token", right_on="instrument_key", how="left", validate="m:1")
    out["Mid"] = np.where(out["BidP1"].notna() & out["AskP1"].notna(), (out["BidP1"] + out["AskP1"])/2.0, out["Ltp"])
    out["Spread"] = np.where(out["BidP1"].notna() & out["AskP1"].notna(), (out["AskP1"] - out["BidP1"]), np.nan)
    out["DepthImb"] = np.where(
        (out["BidQ1"].notna() & out["AskQ1"].notna() & ((out["BidQ1"] + out["AskQ1"]) > 0)),
        (out["BidQ1"] - out["AskQ1"]) / (out["BidQ1"] + out["AskQ1"]),
        np.nan,
    )
    return out

# IV z-score stats (Welford)
from collections import defaultdict
_iv_stats = defaultdict(lambda: {"n":0, "mean":0.0, "M2":0.0})
def update_iv_stats(token: str, iv_value: float):
    if iv_value is None or not np.isfinite(iv_value): return
    s = _iv_stats[token]
    n1 = s["n"] + 1
    delta = iv_value - s["mean"]
    mean = s["mean"] + delta / n1
    delta2 = iv_value - mean
    M2 = s["M2"] + delta * delta2
    s["n"], s["mean"], s["M2"] = n1, mean, M2

def iv_zscore_for(token: str, iv_value: float):
    s = _iv_stats[token]
    if s["n"] < max(IV_Z_MIN_COUNT, 2):
        return 0.0, True
    var = s["M2"] / max(s["n"] - 1, 1)
    std = math.sqrt(max(var, 1e-12))
    z = (iv_value - s["mean"]) / std if std > 0 else 0.0
    return z, (abs(z) <= IV_Z_MAX)

# Streaming globals
live_streamer = None
current_tokens = list(token_list)
spot_ltp_current = spot_ltp_initial
last_center_nearest = int(STRIKE_STEP[UNDERLYING] * math.floor((spot_ltp_initial + STRIKE_STEP[UNDERLYING]/2)/STRIKE_STEP[UNDERLYING]))
_last_recenter_ts = 0.0


In [ ]:

def start_live_stream(tokens: List[str], mode: str = "full_d30"):
    global live_streamer
    if not UPSDK_AVAILABLE: raise RuntimeError("Upstox SDK not installed.")
    if not CredentialUpstox.ACCESS_TOKEN: raise RuntimeError("ACCESS_TOKEN missing.")

    configuration = upstox_client.Configuration()
    configuration.access_token = CredentialUpstox.ACCESS_TOKEN
    api_client = upstox_client.ApiClient(configuration)
    streamer = upstox_client.MarketDataStreamerV3(api_client, instrument_key=tokens, mode=mode)

    def _on_message(msg):
        global df_feed, df_feed_enriched, spot_ltp_current
        feeds = msg.get("feeds", {})
        for token, payload in feeds.items():
            ff = payload.get("fullFeed",{}).get("marketFF",{})
            ltpc = ff.get("ltpc",{})
            level = ff.get("marketLevel",{}).get("bidAskQuote",[{}])
            greeks = ff.get("optionGreeks",{}) or {}
            if token == UNDERLYING_SPOT_TOKEN:
                try:
                    if ltpc.get("ltp") is not None:
                        spot_ltp_current = float(ltpc.get("ltp"))
                except Exception: pass
                continue
            try:
                iv_val = ff.get("iv")
                if iv_val is not None:
                    update_iv_stats(token, float(iv_val))
            except Exception: pass
            row = {
                "Token": token,
                "Ltp": float(ltpc.get("ltp")) if ltpc.get("ltp") is not None else np.nan,
                "Ltq": float(ltpc.get("ltq")) if ltpc.get("ltq") is not None else np.nan,
                "Cp": float(ltpc.get("cp")) if ltpc.get("cp") is not None else np.nan,
                "BidP1": float(level[0].get("bidP")) if level and level[0].get("bidP") is not None else np.nan,
                "BidQ1": float(level[0].get("bidQ")) if level and level[0].get("bidQ") is not None else np.nan,
                "AskP1": float(level[0].get("askP")) if level and level[0].get("askP") is not None else np.nan,
                "AskQ1": float(level[0].get("askQ")) if level and level[0].get("askQ") is not None else np.nan,
                "Ltt": to_ist_ms(ltpc.get("ltt")),
                "Oi": float(ff.get("oi")) if ff.get("oi") is not None else np.nan,
                "Iv": float(ff.get("iv")) if ff.get("iv") is not None else np.nan,
                "Atp": float(ff.get("atp")) if ff.get("atp") is not None else np.nan,
                "Tbq": float(ff.get("tbq")) if ff.get("tbq") is not None else np.nan,
                "Tsq": float(ff.get("tsq")) if ff.get("tsq") is not None else np.nan,
                "Delta": float(greeks.get("delta")) if greeks.get("delta") is not None else np.nan,
                "Theta": float(greeks.get("theta")) if greeks.get("theta") is not None else np.nan,
                "Gamma": float(greeks.get("gamma")) if greeks.get("gamma") is not None else np.nan,
                "Vega":  float(greeks.get("vega"))  if greeks.get("vega")  is not None else np.nan,
                "Rho":   float(greeks.get("rho"))   if greeks.get("rho")   is not None else np.nan,
            }
            with _df_lock:
                if token in df_feed["Token"].values:
                    for k,v in row.items(): df_feed.loc[df_feed["Token"]==token, k] = v
                else:
                    df_feed = pd.concat([df_feed, pd.DataFrame([row])], ignore_index=True)
                df_feed_enriched = enrich_feed(df_feed, df_chain)

    streamer.on_message = _on_message
    streamer.on_open = lambda: print("Market WS opened")
    streamer.on_error = lambda e: print("Market WS error:", e)
    streamer.on_close = lambda: print("Market WS closed")
    streamer.connect()
    live_streamer = streamer
    return streamer

def stop_live_stream():
    global live_streamer
    try:
        if live_streamer is not None:
            if hasattr(live_streamer, "close"): live_streamer.close()
            elif hasattr(live_streamer, "disconnect"): live_streamer.disconnect()
            elif hasattr(live_streamer, "ws"): live_streamer.ws.close()
            print("Market stream stop requested.")
    except Exception as e:
        print("Error stopping stream:", e)
    finally:
        live_streamer = None


In [ ]:

def compute_token_list_for_spot(spot_ltp: float):
    strikes = nearest_strikes_from_spot(spot_ltp, UNDERLYING, span=SPAN_STRIKES)
    df_sel = df_chain[df_chain["strike_price"].isin(strikes)].sort_values(["strike_price","instrument_type"])
    tokens = df_sel["instrument_key"].dropna().astype(str).unique().tolist()
    return tokens

def maybe_recenter_tokens():
    global last_center_nearest, _last_recenter_ts, current_tokens, df_chain_sel
    step = STRIKE_STEP.get(UNDERLYING.upper(), 50)
    nearest = int(step * math.floor((spot_ltp_current + step/2)/step))
    now = time.time()
    if abs(nearest - last_center_nearest) >= step and (now - _last_recenter_ts) >= RECENTER_COOLDOWN_S:
        try:
            new_tokens = compute_token_list_for_spot(spot_ltp_current)
            if not new_tokens: return False
            tokens_with_spot = list(dict.fromkeys(new_tokens + [UNDERLYING_SPOT_TOKEN]))
            if RECENTER_LOG:
                print(f"[RECENTER] spot={spot_ltp_current:.2f}, nearest={nearest}, old_center={last_center_nearest}")
                print(f"[RECENTER] tokens: {len(current_tokens)} → {len(new_tokens)}")
            stop_live_stream(); start_live_stream(tokens_with_spot, mode=WEBSOCKET_MODE)
            last_center_nearest = nearest; _last_recenter_ts = now; current_tokens = new_tokens
            df_chain_sel = df_chain[df_chain["instrument_key"].isin(new_tokens)].sort_values(["strike_price","instrument_type"])
            return True
        except Exception as e:
            print("Recenter failed:", e); traceback.print_exc()
    return False

def recenter_daemon():
    while True:
        try:
            time.sleep(1.0); maybe_recenter_tokens()
        except Exception:
            time.sleep(2.0); continue


In [ ]:

def eligible_entry(row: pd.Series, side: str):
    reasons = []
    delta = row.get("Delta")
    if delta is None or not np.isfinite(delta):
        reasons.append("no_delta")
    else:
        if not (DELTA_MIN <= abs(float(delta)) <= DELTA_MAX):
            reasons.append(f"absΔ={abs(float(delta)):.2f}∉[{DELTA_MIN},{DELTA_MAX}]")
    spread = row.get("Spread")
    if (spread is None) or (not np.isfinite(spread)) or (float(spread) > SPREAD_MAX):
        reasons.append(f"spread={spread} > {SPREAD_MAX}")
    imb = row.get("DepthImb")
    if imb is None or not np.isfinite(imb):
        reasons.append("no_depthimb")
    else:
        imb = float(imb)
        if side.upper() == "BUY" and imb < +DEPTH_IMB_MIN:
            reasons.append(f"imb={imb:.2f} < +{DEPTH_IMB_MIN}")
        if side.upper() == "SELL" and imb > -DEPTH_IMB_MIN:
            reasons.append(f"imb={imb:.2f} > -{DEPTH_IMB_MIN}")
    iv = row.get("Iv")
    z, iv_ok = (0.0, True)
    try:
        if iv is not None and np.isfinite(iv):
            z, iv_ok = iv_zscore_for(row["Token"], float(iv))
            if not iv_ok: reasons.append(f"|z_IV|={abs(z):.2f} > {IV_Z_MAX}")
    except Exception: pass
    ok = len(reasons) == 0
    return ok, ";".join(reasons), {"iv_z": z}

def _compact_row(r: pd.Series):
    return {
        "token": str(r["Token"]),
        "tsym": str(r.get("trading_symbol", "")),
        "mid": float(r.get("Mid", np.nan)),
        "spread": float(r.get("Spread", np.nan)),
        "delta": float(r.get("Delta", np.nan)),
        "gamma": float(r.get("Gamma", np.nan)),
        "theta": float(r.get("Theta", np.nan)),
        "iv": float(r.get("Iv", np.nan)),
        "depthImb": float(r.get("DepthImb", np.nan)),
        "strike": float(r.get("strike_price", np.nan)),
        "type": str(r.get("instrument_type","")),
    }


In [ ]:

def _build_ollama_prompt(context: dict) -> str:
    rules = context.get("rules", {})
    ce, pe = context.get("ce", {}), context.get("pe", {})
    spot = context.get("spot", 0.0)
    return (
        "You are a scalping trade scorer. Return a single JSON object with 'score' in [0,1].\n"
        "No explanation.\n\n"
        f"spot: {spot}\n"
        f"ce: mid={ce.get('mid')}, spread={ce.get('spread')}, delta={ce.get('delta')}, gamma={ce.get('gamma')}, theta={ce.get('theta')}, iv={ce.get('iv')}, depthImb={ce.get('depthImb')}\n"
        f"pe: mid={pe.get('mid')}, spread={pe.get('spread')}, delta={pe.get('delta')}, gamma={pe.get('gamma')}, theta={pe.get('theta')}, iv={pe.get('iv')}, depthImb={pe.get('depthImb')}\n"
        f"rules: absDelta={rules.get('absDelta')}, spreadMax={rules.get('spreadMax')}, depthImbMin={rules.get('depthImbMin')}, ivZMax={rules.get('ivZMax')}\n\n"
        "Return strictly: {\"score\": <float>}"
    )

def _router_log(entry: dict):
    entry = dict(entry); entry.setdefault("t", time.perf_counter())
    ROUTER_LOG.append(entry)
    if len(ROUTER_LOG) > ROUTER_LOG_MAX:
        del ROUTER_LOG[: len(ROUTER_LOG) - ROUTER_LOG_MAX]

def _post_stub(context: dict):
    t0 = time.perf_counter()
    if not LLM_ENDPOINT:
        return 0.6, (time.perf_counter() - t0)*1000.0, False
    try:
        import requests
        r = requests.post(LLM_ENDPOINT, json=context, timeout=LLM_TIMEOUT_S)
        s = float(r.json().get("score", 0.6)) if r.ok else 0.6
        return max(0.0, min(1.0, s)), (time.perf_counter() - t0)*1000.0, True
    except Exception:
        return 0.6, (time.perf_counter() - t0)*1000.0, False

def _post_ollama(context: dict):
    t0 = time.perf_counter()
    if not OLLAMA_MODEL:
        return 0.6, (time.perf_counter() - t0)*1000.0, False
    try:
        import requests
        payload = {"model": OLLAMA_MODEL, "prompt": _build_ollama_prompt(context), "stream": False,
                   "options": {"temperature": 0.1, "num_predict": OLLAMA_NUM_PREDICT}, "format": "json"}
        r = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=OLLAMA_TIMEOUT_S)
        if not r.ok:
            payload.pop("format", None)
            r = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=OLLAMA_TIMEOUT_S)
        txt = r.json().get("response", "") if r.ok else ""
        try:
            s = float(json.loads(txt).get("score", 0.6))
        except Exception:
            m = re.search(r"([01](?:\.\d+)?)", txt)
            s = float(m.group(1)) if m else 0.6
        return max(0.0, min(1.0, s)), (time.perf_counter() - t0)*1000.0, True
    except Exception:
        return 0.6, (time.perf_counter() - t0)*1000.0, False

def score_with_router_and_meta(context: dict):
    thr = LLM_SCORE_THRESHOLD
    stub_score, stub_ms, stub_ok = _post_stub(context)
    stub_decision = (stub_score >= thr)
    route = "no_stub"; ollama_score, ollama_ms, used_ollama = (None, 0.0, False)

    if stub_ok:
        if stub_score >= STUB_UPPER:
            final_score = stub_score; route = "stub_high"
        elif stub_score <= STUB_LOWER:
            final_score = stub_score; route = "stub_low"
        else:
            o_score, o_ms, o_ok = _post_ollama(context)
            used_ollama = True; ollama_score, ollama_ms = (o_score, o_ms)
            if o_ok: final_score = o_score; route = "gray_ollama"
            else:    final_score = stub_score; route = "gray_fallback_stub"
    else:
        o_score, o_ms, o_ok = _post_ollama(context)
        used_ollama = True if o_ok else False; ollama_score, ollama_ms = (o_score, o_ms)
        final_score = o_score if o_ok else 0.6
        route = "no_stub_ollama" if o_ok else "neutral"

    final_decision = (final_score >= thr)
    entry = {"route": route, "stub_score": stub_score, "stub_ms": round(stub_ms, 2),
             "ollama_score": ollama_score, "ollama_ms": round(ollama_ms, 2),
             "final_score": final_score, "stub_decision": stub_decision,
             "final_decision": final_decision, "used_ollama": used_ollama}
    _router_log(entry)
    return float(final_score), entry


In [ ]:

def ask_llm_for_strategy(df_snapshot: pd.DataFrame, use_mock: bool = True) -> Dict[str, Any]:
    if df_snapshot.empty: return {"legs": []}
    snap = df_snapshot.dropna(subset=["Mid","strike_price","instrument_type"]).copy()
    if snap.empty: return {"legs": []}

    snap["dist"] = (snap["Mid"] - snap["strike_price"]).abs()
    def delta_score(d):
        try:
            d = abs(float(d)); return abs(0.5 - d) if np.isfinite(d) else 0.5
        except Exception: return 0.5
    snap["delta_score"] = snap["Delta"].apply(delta_score)

    picks = []
    for opt in ("CE","PE"):
        sub = snap[snap["instrument_type"] == opt].sort_values(["dist","delta_score"]).head(3)
        if sub.empty: continue
        picks.append(sub.iloc[0])
    if len(picks) < 2: return {"legs": []}
    ce_row, pe_row = (picks[0], picks[1]) if picks[0]["instrument_type"]=="CE" else (picks[1], picks[0])

    ce_ok, _, _ = eligible_entry(ce_row, side="SELL")
    pe_ok, _, _ = eligible_entry(pe_row, side="SELL")
    if not ce_ok or not pe_ok: return {"legs": []}

    context = {"spot": float(spot_ltp_current),
               "ce": _compact_row(ce_row),
               "pe": _compact_row(pe_row),
               "position": [],
               "rules": {"absDelta":[DELTA_MIN, DELTA_MAX], "spreadMax":SPREAD_MAX, "depthImbMin":DEPTH_IMB_MIN, "ivZMax":IV_Z_MAX}}
    score, router_meta = score_with_router_and_meta(context)
    if USE_LLM and (score < LLM_SCORE_THRESHOLD): return {"legs": [], "meta": router_meta}

    legs = []
    for r in (ce_row, pe_row):
        lot = int(r.get("lot_size") or 0) or 50
        legs.append({"token": str(r["Token"]), "side": "SELL", "qty": lot, "product": DEFAULT_PRODUCT, "order_type": "LIMIT",
                     "_row": r.to_dict()})
    return {"legs": legs, "meta": {"score": score, **router_meta}}


In [ ]:

open_positions = {}  # token -> {qty, side, avg_price}

# === Multi-entry controls ===
MULTI_ENTRY_PER_TOKEN = True
MAX_OPEN_TRADES_PER_TOKEN = 3
MAX_NET_QTY_PER_TOKEN = 3 * 50  # example if lot=50
ENTRY_COOLDOWN_S = 5.0
ADD_REQUIRES_IMPROVEMENT = True

_last_entry_ts = {}
_last_entry_micro = {}

# --- Ledger ---
TRADE_LOG = []                 # append-only
TRADE_OPEN_STACK = {}          # token -> list[int] (open entry indices)

def _open_count(token: str) -> int:
    return len([i for i in TRADE_OPEN_STACK.get(token, []) if TRADE_LOG[i].get("exit_ts") is None])

def _net_open_qty(token: str) -> int:
    total = 0
    for i in TRADE_OPEN_STACK.get(token, []):
        rec = TRADE_LOG[i]
        if rec.get("exit_ts") is None:
            total += int(rec.get("remaining_qty", rec.get("qty", 0)))
    return total

def _iv_z_for_row(r):
    try:
        z, _ok = iv_zscore_for(r["Token"], float(r.get("Iv", np.nan)))
        return z
    except Exception:
        return 0.0

def _micro_snapshot(r):
    return {"Spread": float(r.get("Spread", np.nan)),
            "DepthImb": float(r.get("DepthImb", np.nan)),
            "IvZ": _iv_z_for_row(r)}

def _micro_improved(token: str, r) -> bool:
    last = _last_entry_micro.get(token)
    now = _micro_snapshot(r)
    _last_entry_micro[token] = now
    if not ADD_REQUIRES_IMPROVEMENT or not last:
        return True
    return (
        (np.isfinite(now["Spread"]) and np.isfinite(last["Spread"]) and now["Spread"] <= last["Spread"]) and
        (np.isfinite(now["DepthImb"]) and np.isfinite(last["DepthImb"]) and now["DepthImb"] <= last["DepthImb"]) and
        (np.isfinite(now["IvZ"]) and np.isfinite(last["IvZ"]) and abs(now["IvZ"]) <= abs(last["IvZ"]) + 1e-9)
    )

def _can_open_more(token: str, add_qty: int) -> (bool, str):
    if not MULTI_ENTRY_PER_TOKEN:
        return (_open_count(token) == 0, "multi_disabled")
    if _open_count(token) >= MAX_OPEN_TRADES_PER_TOKEN:
        return (False, "max_open_trades")
    if (_net_open_qty(token) + add_qty) > MAX_NET_QTY_PER_TOKEN:
        return (False, "net_qty_cap")
    if time.time() - _last_entry_ts.get(token, 0) < ENTRY_COOLDOWN_S:
        return (False, "entry_cooldown")
    return (True, "ok")

def _now_ts():
    return pd.Timestamp.now(tz=IST)

def log_trade_open(token: str, side: str, qty: int, price: float, plan_meta: dict, row_data: dict,
                   target_pct=None, stop_pct=None, order_id: Optional[str]=None):
    micro = _micro_snapshot(row_data or {})
    rec = {
        "token": token, "side": side, "qty": int(qty),
        "remaining_qty": int(qty),
        "entry_price": float(price), "entry_ts": _now_ts(),
        "entry_spread": micro["Spread"], "entry_depthimb": micro["DepthImb"], "entry_iv_z": micro["IvZ"],
        "target_pct": float(target_pct if target_pct is not None else 0.20),
        "stop_pct": float(stop_pct if stop_pct is not None else 0.40),
        "router_route": plan_meta.get("route") if plan_meta else None,
        "score_final": plan_meta.get("final_score") if plan_meta else None,
        "score_stub": plan_meta.get("stub_score") if plan_meta else None,
        "score_ollama": plan_meta.get("ollama_score") if plan_meta else None,
        "stub_ms": plan_meta.get("stub_ms"), "ollama_ms": plan_meta.get("ollama_ms"),
        "order_id": order_id,
        "exit_reason": None, "exit_price": None, "exit_ts": None,
        "pnl_abs": None, "pnl_pct": None, "hold_s": None,
    }
    TRADE_LOG.append(rec)
    idx = len(TRADE_LOG) - 1
    TRADE_OPEN_STACK.setdefault(token, []).append(idx)
    _last_entry_ts[token] = time.time()
    return idx

def log_trade_exit(token: str, reason: str, exit_price: float, match="LIFO", exit_qty=None):
    stack = TRADE_OPEN_STACK.get(token, [])
    if not stack: return None
    idx = stack[-1] if match == "LIFO" else stack[0]
    rec = TRADE_LOG[idx]
    if rec.get("exit_ts") is not None:
        stack.pop(-1 if match=="LIFO" else 0)
        return log_trade_exit(token, reason, exit_price, match, exit_qty)
    rem = int(rec.get("remaining_qty", rec["qty"])); take = rem if (exit_qty is None or exit_qty >= rem) else int(exit_qty)
    new_rem = rem - take
    side = rec["side"].upper(); entry = rec["entry_price"]
    pnl_leg = (exit_price - entry) if side=="BUY" else (entry - exit_price)
    realized_pct = pnl_leg / entry if entry else 0.0
    if new_rem <= 0:
        rec["remaining_qty"] = 0; rec["exit_reason"] = reason; rec["exit_price"] = float(exit_price); rec["exit_ts"] = _now_ts()
        rec["pnl_abs"] = float(pnl_leg); rec["pnl_pct"] = float(realized_pct); rec["hold_s"] = float((rec["exit_ts"] - rec["entry_ts"]).total_seconds())
        stack.pop(-1 if match=="LIFO" else 0)
        return idx
    else:
        rec["remaining_qty"] = new_rem
        return idx


In [ ]:

@dataclass
class ExitConfig:
    dry_run: bool = True
    target_pct: float = 0.2
    stop_pct: float = 0.4
    check_interval_s: float = 1.0

_exit_thread = None; _exit_stop_evt = threading.Event()

def eval_exits_for_token(token: str, row: pd.Series) -> list:
    out = []
    for idx in list(TRADE_OPEN_STACK.get(token, [])):
        rec = TRADE_LOG[idx]
        if rec.get("exit_ts") is not None: continue
        rem = int(rec.get("remaining_qty", rec["qty"]))
        if rem <= 0: continue
        side = rec["side"].upper(); entry = float(rec["entry_price"])
        px = float(row.get("Mid") if np.isfinite(row.get("Mid", np.nan)) else row.get("Ltp"))
        if not np.isfinite(px) or entry <= 0: continue
        pnl_pct = (px - entry)/entry if side=="BUY" else (entry - px)/entry
        if pnl_pct >= rec["target_pct"]:
            out.append({"action":"EXIT","reason":"target","token":token,"qty":rem,"side":"SELL" if side=="BUY" else "BUY","exit_price":px,"idx":idx})
        elif pnl_pct <= -rec["stop_pct"]:
            out.append({"action":"EXIT","reason":"stop","token":token,"qty":rem,"side":"SELL" if side=="BUY" else "BUY","exit_price":px,"idx":idx})
    return out

def _exit_worker(cfg: ExitConfig):
    while not _exit_stop_evt.is_set():
        time.sleep(cfg.check_interval_s)
        with _df_lock:
            snapshot = df_feed_enriched.copy()
        for token, pos in list(open_positions.items()):
            row = snapshot.loc[snapshot["Token"]==token]
            if row.empty: continue
            row = row.iloc[0]
            signals = eval_exits_for_token(token, row)
            for sig in signals:
                # Log PnL immediately (live-first); reconciler will refine via fills if needed
                log_trade_exit(token, sig["reason"], sig["exit_price"], match="LIFO", exit_qty=sig["qty"])
                if cfg.dry_run or (exec_v3 is None):
                    print(f"[EXIT-SIM] {sig}")
                else:
                    # Place live exit via V3 (marketable-limit around BBO)
                    best_bid = float(row.get("BidP1")) if np.isfinite(row.get("BidP1", np.nan)) else float(row.get("Mid", 0.0))
                    best_ask = float(row.get("AskP1")) if np.isfinite(row.get("AskP1", np.nan)) else float(row.get("Mid", 0.0))
                    tick = float(row.get("tick_size") or 0.05)
                    info = exec_v3.place_order_v3(
                        instrument_token=token, side=sig["side"], quantity=int(sig["qty"]),
                        best_bid=best_bid, best_ask=best_ask, tick_size=tick, product=PRODUCT_MAP.get(DEFAULT_PRODUCT,"I"),
                        marketable_limit=True, buffer_ticks=LIMIT_BUFFER_TICKS, tag="LLM-EXIT"
                    )
                    print("[EXIT-LIVE] sent:", info)
            if signals:
                open_positions[token] = {"qty": 0, "side": pos.get("side","SELL"), "avg_price": pos.get("avg_price", 0.0)}

def start_exit_manager(cfg: ExitConfig):
    global _exit_thread, _exit_stop_evt
    _exit_stop_evt = threading.Event()
    _exit_thread = threading.Thread(target=_exit_worker, args=(cfg,), daemon=True)
    _exit_thread.start()

def stop_exit_manager():
    global _exit_thread, _exit_stop_evt
    _exit_stop_evt.set()
    if _exit_thread is not None:
        _exit_thread.join(timeout=5)


In [ ]:

_last_order_ts = 0.0

def _lat_log(event: str, **kwargs):
    ts = time.perf_counter()
    LATENCY_LOG.append({"t": ts, "event": event, **kwargs})
    if len(LATENCY_LOG) > LATENCY_LOG_MAX:
        del LATENCY_LOG[: len(LATENCY_LOG) - LATENCY_LOG_MAX]



## Upstox **V3** Execution & Live Reconciliation


In [ ]:

UPSTOX_DOCS_BASE = 'https://upstox.com/developer/api-documentation/orders'
UPSTOX_WS_DOCS = 'https://upstox.com/developer/api-documentation/get-portfolio-stream-feed/'
VERBOSE_EXEC  = True
VERBOSE_RECON = True

class UpstoxV3Exec:
    def __init__(self, access_token: str):
        self.access_token = access_token
        configuration = upstox_client.Configuration(); configuration.access_token = self.access_token
        self._api_client = upstox_client.ApiClient(configuration)
        self.order_api_v3 = upstox_client.OrderApiV3(self._api_client)

    @staticmethod
    def _now_ms():
        return int(time.time() * 1000)

    def marketable_limit(self, best_bid: float, best_ask: float, side: str, tick_size: float, buffer_ticks: int = 1) -> float:
        px = (best_ask + buffer_ticks * tick_size) if side.upper()=="BUY" else (best_bid - buffer_ticks * tick_size)
        ticks = round(px / max(tick_size, 1e-6))
        return float(ticks * max(tick_size, 1e-6))

    def place_order_v3(self, *, instrument_token: str, side: str, quantity: int,
                       best_bid: float, best_ask: float, price: float = None,
                       product: str = "I", validity: str = "DAY", disclosed_quantity: int = 0,
                       trigger_price: float = 0.0, is_amo: bool = False,
                       marketable_limit: bool = True, buffer_ticks: int = 1,
                       tick_size: float = 0.05, tag: str = None) -> dict:
        if tag is None:
            tag = f"router-{uuid.uuid4().hex[:8]}"
        if price is None and marketable_limit:
            price = self.marketable_limit(best_bid, best_ask, side, tick_size, buffer_ticks)
        req = upstox_client.PlaceOrderV3Request(
            quantity=int(quantity), product=product, validity=validity, price=float(price if price is not None else 0.0),
            tag=tag, instrument_token=instrument_token,
            order_type='LIMIT' if price else 'MARKET', transaction_type=side.upper(),
            disclosed_quantity=int(disclosed_quantity), trigger_price=float(trigger_price), is_amo=bool(is_amo), slice=False
        )
        t0 = self._now_ms()
        try:
            resp = self.order_api_v3.place_order(req)
            t1 = self._now_ms()
            order_id = None
            if hasattr(resp, "data") and resp.data:
                if isinstance(resp.data, dict) and "order_id" in resp.data: order_id = resp.data["order_id"]
                elif isinstance(resp.data, dict) and "order_ids" in resp.data: 
                    order_ids = resp.data.get("order_ids") or []; order_id = order_ids[0] if order_ids else None
            info = {"ok": True, "order_id": order_id, "sent_ts": t0, "ack_ts": t1, "broker_latency_ms": (t1 - t0)}
            if VERBOSE_EXEC: print("[V3/place] ok", info)
            return info
        except ApiException as e:
            t1 = self._now_ms()
            err = {"ok": False, "error": str(e), "sent_ts": t0, "ack_ts": t1}
            print("[V3/place] ERROR", err); return err

exec_v3 = UpstoxV3Exec(CredentialUpstox.ACCESS_TOKEN) if (UPSDK_AVAILABLE and CredentialUpstox.ACCESS_TOKEN) else None


In [ ]:

from collections import defaultdict

FILL_LOG_COLS = ['ts', 'order_id', 'status', 'filled_qty', 'avg_price', 'trading_symbol', 'instrument_token', 'transaction_type', 'raw']
FILL_LOG = pd.DataFrame(columns=FILL_LOG_COLS)

class UpstoxPortfolioReconciler:
    def __init__(self, access_token: str):
        configuration = upstox_client.Configuration(); configuration.access_token = access_token
        api_client = upstox_client.ApiClient(configuration)
        self.streamer = upstox_client.PortfolioDataStreamer(api_client, order_update=True, position_update=False, holding_update=False, gtt_update=False)
        self.order_meta = {}; self.last_seen_fill_qty = defaultdict(int)

    def attach_meta(self, order_id: str, meta: dict):
        self.order_meta[order_id] = meta or {}

    def _append_fill_log(self, record: dict):
        global FILL_LOG
        FILL_LOG.loc[len(FILL_LOG)] = [
            record.get('ts'), record.get('order_id'), record.get('status'), record.get('filled_quantity'),
            record.get('average_price'), record.get('trading_symbol'), record.get('instrument_token'),
            record.get('transaction_type'), json.dumps(record)
        ]

    @staticmethod
    def _extract_update(message) -> dict:
        try:
            data = message
            if isinstance(message, (bytes, str)): data = json.loads(message)
        except Exception: data = message
        d = data.get('data') if isinstance(data, dict) and isinstance(data.get('data'), dict) else (data if isinstance(data, dict) else {})
        flat = {
            'ts': d.get('order_timestamp') or d.get('exchange_timestamp') or int(time.time()*1000),
            'order_id': d.get('order_id') or d.get('id') or d.get('orderId'),
            'status': (d.get('status') or '').lower(),
            'filled_quantity': d.get('filled_quantity') or d.get('filledQuantity') or d.get('quantity') or 0,
            'average_price': d.get('average_price') or d.get('avg_price') or d.get('averagePrice') or None,
            'trading_symbol': d.get('trading_symbol') or d.get('tradingsymbol') or d.get('symbol'),
            'instrument_token': d.get('instrument_token') or d.get('instrumentKey') or d.get('instrument'),
            'transaction_type': d.get('transaction_type') or d.get('side') or d.get('transactionType')
        }
        return flat

    def _on_message(self, message):
        upd = self._extract_update(message)
        oid = upd.get('order_id'); 
        if not oid: 
            if VERBOSE_RECON: print("[recon] no order_id:", message); 
            return
        prev = self.last_seen_fill_qty.get(oid, 0); cur = int(upd.get('filled_quantity') or 0)
        self._append_fill_log(upd)
        if cur > prev and VERBOSE_RECON: print(f"[recon] FILL {oid} {prev}->{cur} @ {upd.get('average_price')} status={upd.get('status')}")
        self.last_seen_fill_qty[oid] = max(prev, cur)
        cb = globals().get("on_upstox_order_update")
        if callable(cb):
            try: cb(upd, self.order_meta.get(oid, {}))
            except Exception as e: 
                if VERBOSE_RECON: print("[ledger-hook] error:", e)

    def start(self):
        self.streamer.on('message', self._on_message)
        self.streamer.auto_reconnect(True, 5, 20)
        self.streamer.connect()

recon = UpstoxPortfolioReconciler(CredentialUpstox.ACCESS_TOKEN) if (UPSDK_AVAILABLE and CredentialUpstox.ACCESS_TOKEN) else None

def start_reconciler():
    if recon is None:
        print("Reconciler not initialized (SDK/token missing)."); return
    print("Starting Portfolio Stream Feed reconciler…"); recon.start()


In [ ]:

def on_upstox_order_update(update: dict, meta: dict):
    oid = update.get("order_id")
    status = (update.get("status") or "").lower()
    avg_px = update.get("average_price")
    token = meta.get("token") or update.get("instrument_token")
    entry_idx = meta.get("entry_idx")
    if entry_idx is not None and 0 <= entry_idx < len(TRADE_LOG):
        rec = TRADE_LOG[entry_idx]
        if rec.get("exit_ts") is None and rec.get("entry_price") and avg_px and status in ("complete","completed"):
            try: rec["entry_price"] = float(avg_px)
            except Exception: pass
    return True


In [ ]:

def place_orders(plan: Dict[str, Any], df_enriched: pd.DataFrame, dry_run: bool = True) -> List[Dict[str, Any]]:
    results = []
    if not plan or "legs" not in plan:
        return results
    plan_meta = plan.get("meta", {})

    global _last_order_ts
    for leg in plan["legs"]:
        token = str(leg["token"])
        row = df_enriched.loc[df_enriched["Token"]==token]
        if row.empty:
            results.append({"status":"rejected","reason":"token_not_found","leg":leg}); continue
        row = row.iloc[0]
        lot = int(row.get("lot_size") or 0); qty = int(leg.get("qty", 0))
        if lot and qty % lot != 0:
            results.append({"status":"rejected","reason":f"qty_not_multiple_of_lot({lot})","leg":leg}); continue
        if qty <= 0 or qty > MAX_QTY_PER_LEG:
            results.append({"status":"rejected","reason":"qty_bounds","leg":leg}); continue

        ok_open, reason = _can_open_more(token, qty)
        if not ok_open:
            results.append({"status":"rejected","reason":f"multi_entry_gate:{reason}","leg":leg}); continue
        if ADD_REQUIRES_IMPROVEMENT and not _micro_improved(token, row):
            results.append({"status":"rejected","reason":"micro_not_improved","leg":leg}); continue

        side = leg["side"].upper()
        product_code = PRODUCT_MAP.get(leg.get("product", DEFAULT_PRODUCT), PRODUCT_MAP[DEFAULT_PRODUCT])
        best_bid = float(row.get("BidP1")) if np.isfinite(row.get("BidP1", np.nan)) else float(row.get("Mid", 0.0))
        best_ask = float(row.get("AskP1")) if np.isfinite(row.get("AskP1", np.nan)) else float(row.get("Mid", 0.0))
        tick = float(row.get("tick_size") or 0.05)

        now = time.perf_counter()
        delay_ms = ORDER_MIN_GAP_MS - (now - _last_order_ts) * 1000.0
        if delay_ms > 0: time.sleep(delay_ms / 1000.0)
        _last_order_ts = time.perf_counter()
        _lat_log("order_send", token=token, side=side, qty=qty, order_type="LIMIT")

        if dry_run or (exec_v3 is None):
            fill = float(row.get("Mid")) if np.isfinite(row.get("Mid", np.nan)) else float(row.get("Ltp", 0.0))
            results.append({"status":"simulated","token":token,"qty":qty,"side":side,"product":product_code,
                            "order_type":"LIMIT","limit_price":fill, "fill_price":fill, "router_meta": plan_meta})
            pos = open_positions.get(token, {"qty": 0, "side": side, "avg_price": 0.0})
            new_qty = pos["qty"] + qty if side == pos["side"] else pos["qty"] - qty
            new_avg = (pos["avg_price"]*pos["qty"] + fill*qty)/max(new_qty,1) if new_qty>0 and side==pos["side"] else fill
            open_positions[token] = {"qty": max(new_qty,0), "side": side, "avg_price": new_avg}
            log_trade_open(token, side, qty, fill, plan_meta, leg.get("_row", {}), order_id=None)
            _lat_log("order_ack", token=token)
        else:
            info = exec_v3.place_order_v3(
                instrument_token=token, side=side, quantity=qty,
                best_bid=best_bid, best_ask=best_ask, tick_size=tick,
                product=PRODUCT_MAP.get(DEFAULT_PRODUCT,"I"), validity="DAY",
                marketable_limit=True, buffer_ticks=LIMIT_BUFFER_TICKS, tag="LLM-ENTRY"
            )
            if not info.get("ok"):
                results.append({"status":"rejected","reason":"v3_place_error","detail":info}); continue
            order_id = info.get("order_id")
            fill = float(row.get("Mid") or row.get("Ltp") or 0.0)
            pos = open_positions.get(token, {"qty": 0, "side": side, "avg_price": 0.0})
            new_qty = pos["qty"] + qty if side == pos["side"] else pos["qty"] - qty
            new_avg = (pos["avg_price"]*pos["qty"] + fill*qty)/max(new_qty,1) if new_qty>0 and side==pos["side"] else fill
            open_positions[token] = {"qty": max(new_qty,0), "side": side, "avg_price": new_avg}
            entry_idx = log_trade_open(token, side, qty, fill, plan_meta, leg.get("_row", {}), order_id=str(order_id))
            if recon is not None and order_id:
                recon.attach_meta(order_id, {"entry_idx": entry_idx, "token": token, "side": side, "qty": qty,
                                             "sent_ts": info.get("sent_ts"), "ack_ts": info.get("ack_ts"), "tag": info.get("tag")})
            results.append({"status":"placed","order_id":order_id,"token":token,"qty":qty,"side":side,"limit_price":None, "router_meta": plan_meta})
            _lat_log("order_ack", token=token)

    return results


In [ ]:

def validate_strategy(plan: Dict[str, Any], df_enriched: pd.DataFrame) -> Optional[Dict[str, Any]]:
    if not plan or not plan.get("legs"): return None
    if len(plan["legs"]) > MAX_OPEN_LEGS: return None
    for leg in plan["legs"]:
        t = str(leg["token"])
        if df_enriched.loc[df_enriched["Token"]==t].empty: return None
    return plan


In [ ]:

if SIMULATION_MODE:
    raise RuntimeError("This notebook is live-first. Set SIMULATION_MODE=False to proceed.")

tokens_with_spot = list(dict.fromkeys(token_list + [UNDERLYING_SPOT_TOKEN]))
streamer = start_live_stream(tokens_with_spot, mode=WEBSOCKET_MODE)
time.sleep(3.0)

with _df_lock:
    snapshot = df_feed_enriched.copy()
print("Feed snapshot rows:", len(snapshot))
display(snapshot.tail(6))

# Start reconciler (non-blocking)
try:
    if RECONCILE_LIVE_PNL: start_reconciler()
except Exception as _e:
    print("Reconciler start skipped:", _e)

def latency_report(n_tail: int = 50) -> pd.DataFrame:
    return pd.DataFrame(LATENCY_LOG[-n_tail:])

def router_report(n_tail: int = 200):
    tail = ROUTER_LOG[-n_tail:]
    df = pd.DataFrame(tail)
    if df.empty: return df, {}
    agree = (df["stub_decision"] == df["final_decision"]).mean()
    used_ollama_rate = df["used_ollama"].mean()
    avg_stub = df["stub_ms"].mean()
    avg_ollama = df.loc[df["used_ollama"], "ollama_ms"].mean() if (df["used_ollama"].any()) else float("nan")
    routes = df["route"].value_counts().to_dict()
    summary = {"n": int(len(df)), "agreement_rate": float(round(agree, 4)), "used_ollama_rate": float(round(used_ollama_rate, 4)),
               "avg_stub_ms": float(round(avg_stub, 2)), "avg_ollama_ms": float(round(avg_ollama, 2)) if avg_ollama == avg_ollama else None, "routes": routes}
    return df, summary

_lat_log("decision_start", rows=len(snapshot))
plan = ask_llm_for_strategy(snapshot, use_mock=True)  # router inside
_lat_log("decision_end", legs=len(plan.get("legs", [])))

try:
    _lat_log("validate_start")
    validated = validate_strategy(plan, snapshot)
    _lat_log("validate_end", ok=True)
except Exception as e:
    validated = None; _lat_log("validate_end", ok=False, err=str(e)); print("Validation failed:", e)

orders = []
if validated:
    _lat_log("place_start", nlegs=len(validated["legs"]))
    orders = place_orders(validated, snapshot, dry_run=(not ORDERS_LIVE))
    _lat_log("place_end", norders=len(orders))
    print("Order results:"); print(json.dumps(orders, indent=2))

cfg = ExitConfig(dry_run=(not EXIT_MANAGER_LIVE))
start_exit_manager(cfg)

_recenter_thread = threading.Thread(target=recenter_daemon, daemon=True); _recenter_thread.start()

print("Live scalper pipeline running. Use the Stop cell to close sockets and exit manager.")


In [ ]:

with _df_lock:
    mon = df_feed_enriched.copy()
display(mon.tail(12))

if not mon.empty:
    mon["dist"] = (mon["Mid"] - mon["strike_price"]).abs()
    picks = mon.sort_values(["dist"]).groupby("instrument_type").head(2)
    rows = []
    for _, r in picks.iterrows():
        ok_sell, why_sell, ex = eligible_entry(r, side="SELL")
        rows.append({"tsym": r.get("trading_symbol",""), "type": r.get("instrument_type",""), "mid": r.get("Mid"),
                     "spread": r.get("Spread"), "depthImb": r.get("DepthImb"), "delta": r.get("Delta"),
                     "iv": r.get("Iv"), "ok_sell": ok_sell, "why_sell": why_sell, "iv_z": ex.get("iv_z")})
    display(pd.DataFrame(rows))
else:
    print("No feed yet.")

df_router, router_summary = router_report(500)
display(df_router.tail(10))
print("Router summary:", router_summary)


In [ ]:

def router_performance_report():
    df = pd.DataFrame([r for r in TRADE_LOG if r.get("exit_ts") is not None])
    if df.empty:
        return df, {}, pd.DataFrame(), {}
    df["hit"] = (df["exit_reason"]=="target").astype(int)
    df["pnl_pct"] = df["pnl_pct"].astype(float)
    grp = df.groupby("router_route", dropna=False)

    def _agg_expect(g):
        win = g[g["pnl_pct"]>0]["pnl_pct"].mean()
        loss = (-g[g["pnl_pct"]<0]["pnl_pct"]).mean()
        win_rate = (g["pnl_pct"]>0).mean()
        exp = g["pnl_pct"].mean()
        return pd.Series({
            "n": len(g),
            "hit_rate": g["hit"].mean(),
            "win_rate": win_rate,
            "avg_win_pct": win if pd.notna(win) else 0.0,
            "avg_loss_pct": loss if pd.notna(loss) else 0.0,
            "expectancy_pct": exp,
            "volatility_pct": g["pnl_pct"].std(),
            "avg_hold_s": g["hold_s"].mean()
        })

    perf = grp.apply(_agg_expect).reset_index()

    def _max_drawdown(series):
        if series.empty: return 0.0
        cum = series.cumsum()
        peak = cum.cummax()
        dd = (cum - peak)
        return float(dd.min())

    df_sorted = df.sort_values("exit_ts")
    df_sorted["pnl_pct"].fillna(0.0, inplace=True)
    overall_mdd = _max_drawdown(df_sorted["pnl_pct"])
    route_mdd = df_sorted.groupby("router_route")["pnl_pct"].apply(_max_drawdown).to_dict()

    dd_summary = {"overall_max_drawdown_pct": overall_mdd, "per_route_max_drawdown_pct": route_mdd}
    return df, perf, df_sorted, dd_summary

def _time_bucket(ts):
    if pd.isna(ts): return "unknown"
    t = ts.tz_convert(IST).time() if ts.tzinfo else ts.tz_localize(IST).time()
    if t >= pd.to_datetime("09:15").time() and t < pd.to_datetime("10:00").time():
        return "open"
    if t >= pd.to_datetime("14:30").time() and t <= pd.to_datetime("15:30").time():
        return "close"
    return "mid"

def _iv_regime(z):
    if pd.isna(z): return "unknown"
    a = abs(float(z))
    if a <= 1.0: return "stable"
    if a <= 2.0: return "elevated"
    return "spike"

def regime_report():
    df = pd.DataFrame([r for r in TRADE_LOG if r.get("exit_ts") is not None])
    if df.empty:
        return pd.DataFrame()
    df["time_bucket"] = df["entry_ts"].apply(_time_bucket)
    df["iv_regime"] = df["entry_iv_z"].apply(_iv_regime)
    df["hit"] = (df["exit_reason"]=="target").astype(int)
    grp = df.groupby(["router_route","time_bucket","iv_regime"], dropna=False)
    out = grp.agg(n=("hit","count"),
                  hit_rate=("hit","mean"),
                  expectancy_pct=("pnl_pct","mean"),
                  volatility_pct=("pnl_pct","std")).reset_index().sort_values(["router_route","time_bucket","iv_regime"])
    return out

df_trades, perf, df_sorted, dd = router_performance_report()
display(df_trades.tail(10))
print("Performance by route:"); display(perf)
print("Drawdown summary:", dd)

regime_perf = regime_report()
print("Regime segmentation (route × time × IV-regime):"); display(regime_perf)


In [ ]:

stop_exit_manager()
stop_live_stream()
print("Shutdown requested.")
